# Maestría en Ciencia de Datos, ITAM

**Curso de Optimización 2 2021-1,"Optimización Avanzada"
Prof. Erick Palacios Moreno**

>*Equipo 5:  
MIGUEL LOPEZ  
CARLOS LOPEZ  
JOSÉ ZARATE  

# PERFILAMIENTO DE CÓDIGO.

**INTRO**

EXPLICAR LA REIMPLEMENTACIÓN QUE SE DESARROLLO 

El paquete que desarrollamos se compone de dos métodos de optimización convexa, uno metaheurístico Hill Climbing y uno por descenso en gradiente Conjugate Gradient. En este notebook realizaremos el perfilamiento de tales métodos, Hill Climbing aplicado para resolver el problema del viajero (TSP) por medio del análisis de la ruta más corta por distancia euclideana.

Nota: el desarrollo de este notebook está basado en la secuencia de perfilamiento utilizada por el Profesor Erick Palacios Moreno en sus notas de clase (ver referencias en README).

-----

## RESOLVER EL PROBLEMA DE TRAVELING SALESMAN PROBLEM (A.K.A. TSP) PARA DISTINTAS CIUDADES UTILIZANDO EL MÉTODO DE HILL CLIMBING

* Distancia Euclideana.
> Dataset: National Traveling Salesman Problems, CANADA

Para ello, realizaremos una ejecución para segemento del dataset con 100 ciudades:

______

**Hill Climbing**  

.

______

**MODIFICACIONES DERIVADAS DE LA ETAPA DE EXPERIMENTACIÓN:**

.

-----

## PERFILAMIENTO

> **Descripción de la instancia**

Hemos decidido utilizar una máquina AWS EC2 m5.2xlarge que es la de mayor capacidad disponible con el programa AWS EDUCATE. Sus características son:

In [1]:
%%bash
lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          8
On-line CPU(s) list:             0-7
Thread(s) per core:              2
Core(s) per socket:              4
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           85
Model name:                      Intel(R) Xeon(R) Platinum 8175M CPU @ 2.50GHz
Stepping:                        4
CPU MHz:                         3120.058
BogoMIPS:                        5000.00
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       128 KiB
L1i cache:                       128 KiB
L2 cache:                        4 MiB
L3 cache:                        33 MiB
NUMA node0 CPU(s):               0-7

In [2]:
%%bash
sudo lshw -C memory

  *-firmware
       description: BIOS
       vendor: Amazon EC2
       physical id: 0
       version: 1.0
       date: 10/16/2017
       size: 64KiB
       capacity: 64KiB
       capabilities: pci edd acpi virtualmachine
  *-memory
       description: System memory
       physical id: 1
       size: 31GiB


Como podemos observar, tenemos disponibles 8 cores 30 gigas de memoria.

Ahora mostraremos las características del kernel utilizado.

In [3]:
%%bash
uname -ar #r for kernel, a for all

Linux ip-10-0-0-123 5.4.0-1047-aws #49-Ubuntu SMP Wed Apr 28 22:47:04 UTC 2021 x86_64 x86_64 x86_64 GNU/Linux


* **Nota:** Como bien sabemos, no es una buena práctica realizar el análisis de perfilamiento de código de nuestros métodos o paquetes desarrollados utilizando _Jupyter notebooks_ debido a que se encapsulan procesos propios del _Jupyter_ y no aisla específicamente los procesos ejecutados del código perfilado. La sugerencia es realizarlo en un intérprete (nuestro caso de _Python_). Por temas académicos, en esta ocasión se desarrollará en el notebook.

____

En primer término evaluaremos el tiempo de ejecución que le toma a nuestro código desarrollado en práctica 2.1 con las reimplementaciones realizadas previamente.


Importaremos nuestro paquete alojado en [Dockerhub](https://hub.docker.com) en la imágen lobolc/Opt_HC_CG:0.1. Para ello utilizaremos el repositorio de desarollo de GitHub de nuestra practica anterior.
**NOTA PARA MI** Definir si el párrafo anterior se indica sólo en este punto o en un README.


#### DISEÑO DEL _TESTING_  
Debido a la naturaleza de nuestro método (metaheurístico), encontrar una solución ópitma global puede resultar imposible conforme se incrementan el número de nodos-ciudad a evaluar para determinar la ruta más corta en el _TSP_. Por este motivo, debemos precisar que nuestro método obtiene la ruta óptima global con un subconjunto de hasta 17 ciudades y aproximaciones menores a un 10% en comparación con la paquetería de OR-tools que utilizaremos para nuestros _test_.

Por lo tanto, para poder realizar nuestro _testing_ hemos decidido utilizar como métrica lo siguiente:
> 1 - la proporción de la distancia obtenida por nuestro método dividida por la distancia obtenida con el método de OR-tools de GOOGLE, de tal forma que un valor cercano a 0 nos indica que ambas rutas son aproximadamente iguales, con una tolerancia de 1e-1.

**Nota:** es importante indicar que la intención del test no es evaluar que se encuentre la misma ruta, si no que la mejor ruta encontrada sea relativamente cercana entre nuestra implementación y la paquetería de comparación.

Packages and libraries required:

In [4]:
import math
import time
import os
import numpy as np
import pandas as pd

Realizar _Import_ de nuestro paquete con el método implementado
>**Nota:** en este caso utilizaremos las funciones integradas en nuestro módulo hill_final.py que contiene la reimplementación que fue necesario realizar sobre nuestro método para corregir un Bug (@Mayo 12, 2021).

In [5]:
pth = os.getcwd()
os.chdir("../")
import notebooks.hill_final as hc
os.chdir(pth)

_Test_ packages requirements

In [6]:
from pytest import approx
import google_or_tools_tsp as tsp

Extraeremos y aplicaremos algunas transformaciones a nuestro dataset:

In [7]:
raw_data = pd.read_csv("../datasets/ca4663.tsp", sep = " ", names = ['index','x','y'])

Para efectos de esta práctica, analizaremos un ejemplo didáctico práctico con 20 ciudades para obtener, en un tiempo razonable para la tolerancia y _number of restarts_, una aproximación consistente tanto con nuestro método con el paquete de OR-tools

In [8]:
raw_data1 = raw_data.drop(['index'], axis = 1)
raw_data2 = raw_data1.dropna()
tsp_cities = raw_data2.iloc[0:17,].to_numpy()

En nuestra función _opt.best_solution()_ desarrollamos el método de _Hill Climbing_ para la determinación de la ruta óptima y la distancia menor al resolver el problema de _Traveling Salesman Problem_ (TSP). También se incluyo la función time del paquete time time que se utiliza en el perfilamiento. Está constituida por cuatro subfunciones a las que se llama durante la ejecución:

* distance_matrix(coordinate): calcula la matriz de distancias (distancia euclideana).  
* random_solution(matrix, initial_point): constuye una ruta aleatoria con los nodos-ciudad  
* calculate_distance(matrix, solution): calcula la distancia entre nodos-ciudad  
* neighbors(matrix, solution): construye todas las posibles rutas del "vecindario" en función de la cercanía de los nodos-ciudad de la _random solution_ propuesta y selecciona la mejor ruta (menor distancia).
* best_solution(coordinate, initial_point = 0, tolerance = 1e-7): es la función principal con la que analizamos un subconjunto del espacio de soluciones y selecciona la mejor ruta posible para el número de subrutinas especificado (número de veces que se reinicia la búsqueda de la mejor ruta a partir de una nueva _random solution_) y   




----

> **Medición de tiempo, Módulo: time**


======= Medición de tiempo =======

In [9]:
start_time = time.time()

# hc.best_solution(order of parameters: dataset, initial point, tolerance and number of restart)
hc_best_dist, hc_best_path, hc_exec_time = hc.best_solution(tsp_cities, 0 , 1e-9, 300)

end_time = time.time()
secs = end_time-start_time

print("La implementación de nuestro método indica que la mejor ruta ecncontrada es:\n", hc_best_path, "\n con una distancia óptima de:\n", hc_best_dist)
print("\nTiempo de ejecución:",secs,"segundos" )

La implementación de nuestro método indica que la mejor ruta ecncontrada es:
 [0, 3, 2, 4, 7, 16, 14, 9, 10, 8, 15, 11, 13, 12, 6, 5, 1, 0] 
 con una distancia óptima de:
 3284.2634919757206

Tiempo de ejecución: 2.6138317584991455 segundos


======= Calcular Objetivo (Google OR-tools) =======

In [10]:
tsp_sol = tsp.main(tsp_cities)

obj = 1 - hc_best_dist/tsp_sol

Objective: 3316
Route:
 0 -> 3 -> 8 -> 10 -> 4 -> 2 -> 9 -> 7 -> 14 -> 16 -> 15 -> 11 -> 13 -> 12 -> 6 -> 5 -> 1 -> 0



======= Validar la solución =======

In [27]:
print(obj == approx(0, abs=1e-1))

True


> **Medición de tiempo, Comando de _magic_: %time**

In [41]:
%time hc.best_solution(tsp_cities, 0 , 1e-9, 300)

CPU times: user 2.55 s, sys: 19 µs, total: 2.55 s
Wall time: 2.54 s


(3339.391869957482,
 [0, 3, 4, 2, 7, 16, 14, 9, 10, 8, 15, 11, 13, 12, 6, 1, 5, 0],
 2.5445618629455566)

Con lo anterior, identificamos lo siguiente:
   * 2.58 segundos del proceso fueron para funciones no relacionadas con el _kernel_ del sistema (procesos de alojamiento, lectura y escritura de variables en memoria, I/O de disco, networking, etc).
   * 0 nanosegundos se utilizaron para funciones a nivel _kernel_ del sistema.

También observamos que el tiempo de ejecución (_wall clock_ o _elapsed time_) desde el inicio de los statements hasta su finalización es exactamente los 2.58 segundos que coinciden con _user_ en este caso.

Al no existir diferencia notable entre _total_ y _wall time_, se observa que el tiempo de ejecución no se ocupó en tareas que no involucran a sys o a user.

Se observó que al volver a ejecutar el proceso anterior, existe variación en las mediciones, por lo que se dificultó realizar las observaciones anteriores. Con el objetivo de disminuir tal variación, utilizarmos la funcionalidad de Timeit (realizar mediciones un número repetido de veces).

> **Medición de tiempo, Comando de _Timeit_: %timeit**

Ejecutaremos este comando para evaluar nuestro método, considerando promediar los tiempos de n=7 ejecuciones calculando su desviación estándar. Este proceso se ejecutará r=11 veces para reportar el mejor resultado.    

In [42]:
%timeit -n 7 -r 11 hc.best_solution(tsp_cities, 0 , 1e-9, 300)

2.6 s ± 12.2 ms per loop (mean ± std. dev. of 11 runs, 7 loops each)


Concluimos que el tiempo promedio de ejecución de nuestro método es de 2.6 segundos con una desviación estándar de 12.2 milisegundos, que es comparable con los 2.54segundos de _%time_.

> **Medición de tiempo, _cProfile_**

In [43]:
import cProfile

In [44]:
cprof = cProfile.Profile()
cprof.enable()
tsp_hc = hc.best_solution(tsp_cities, 0 , 1e-9, 300)
cprof.disable()
cprof.print_stats(sort='cumtime') # buscamo analizar donde se concentre el mayor tiempo acumulado de nuestra func.

         1700243 function calls (1699954 primitive calls) in 2.850 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    2.850    1.425 interactiveshell.py:3400(run_code)
        2    0.000    0.000    2.850    1.425 {built-in method builtins.exec}
        1    0.000    0.000    2.850    2.850 <ipython-input-44-2aa36a8b218e>:3(<module>)
        1    0.015    0.015    2.850    2.850 hill_final.py:94(best_solution)
     3752    0.217    0.000    2.814    0.001 hill_final.py:64(neighbors)
   398013    2.508    0.000    2.532    0.000 hill_final.py:48(calculate_distance)
   393960    0.035    0.000    0.035    0.000 {method 'copy' of 'list' objects}
   467235    0.030    0.000    0.030    0.000 {built-in method builtins.len}
   399366    0.028    0.000    0.028    0.000 {method 'append' of 'list' objects}
      301    0.004    0.000    0.014    0.000 hill_final.py:25(random_solution)
     4816    0.00

Notemos que al haber realizado el perfilamiento con cProfile, nuestro tiempo de ejecución promedio de 2.6 segundos se incremento a 2.85 segundos, lo cual es normal al utilizar esta funcionalidad.  
En este caso, nos enfocaremos en analizar los procesos en los que se realiza una mayor inversión de tiempo:
* Nuestra función calculate_distance fue llamada 398,013 con un tiempo de _tottime_ 2.508 segundos (correspondiente a la línea 48 de nuestro módulo).
* Se tienen otros tres procesos independientes (tottime=cumtime) que implican la mayor cantidad de tiempo de ejecución: method 'copy' of 'list' objects que tiene 393,960 llamadas con 0.035 segundos; built-in method builtins.len con 467,235 llamadas y 0.030 segundos; method 'append' of 'list' objects con 399,366 llamads y 0.028 seugundos.
* Nuestra función (neighbors, linea 64 de nuestro módulo) fue invocada 3,752 veces con un tiempo de ejecución _tottime_ .217 segundos. Es importante observar que en este caso tenemos un _cumtime_ de 2.814 segundos, que incluye los 2.6 segundos de las funciones anteriores y algunas otras de menor tiempo de procesamiento.

Con el objetivo de poder realizar un análisis simplificado de la información anterior, utilizaremos _pstats_ de la clase _Stats_ para un resumen más accesible. En este caso, buscaremos observar las llamadas a method y built.in methods:

In [50]:
cprof.dump_stats("tsp_hc_stats")
import pstats
p_hc_stats = pstats.Stats("tsp_hc_stats")

In [51]:
print(p_hc_stats.sort_stats("cumulative").print_stats("method|built-in"))

Fri May 14 04:25:26 2021    tsp_hc_stats

         1700243 function calls (1699954 primitive calls) in 2.850 seconds

   Ordered by: cumulative time
   List reduced from 54 to 18 due to restriction <'method|built-in'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    2.850    1.425 {built-in method builtins.exec}
   393960    0.035    0.000    0.035    0.000 {method 'copy' of 'list' objects}
   467235    0.030    0.000    0.030    0.000 {built-in method builtins.len}
   399366    0.028    0.000    0.028    0.000 {method 'append' of 'list' objects}
  579/290    0.001    0.000    0.003    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
     5117    0.001    0.000    0.001    0.000 {method 'remove' of 'list' objects}
     7543    0.001    0.000    0.001    0.000 {method 'getrandbits' of '_random.Random' objects}
     4816    0.000    0.000    0.000    0.000 {method 'bit_length' of 'int' objects}
     1875 

Mostraremos ahora cuántas llamadas a funciones _built-in_ se realizaron y cuáles funciones están realizando llamadas a otras.

Esto es de suma importancia, pues nos permite visualizar las posibles dependencias y _bottlenecks_ que podrían existir en nuestro código.

In [52]:
print(p_hc_stats.prim_calls)

1699954


In [53]:
p_hc_stats.strip_dirs().sort_stats("cumulative").print_callers()

   Ordered by: cumulative time

Function                                                                 was called by...
                                                                             ncalls  tottime  cumtime
interactiveshell.py:3400(run_code)                                       <- 
{built-in method builtins.exec}                                          <-       2    0.000    2.850  interactiveshell.py:3400(run_code)
<ipython-input-44-2aa36a8b218e>:3(<module>)                              <-       1    0.000    2.850  {built-in method builtins.exec}
hill_final.py:94(best_solution)                                          <-       1    0.015    2.850  <ipython-input-44-2aa36a8b218e>:3(<module>)
hill_final.py:64(neighbors)                                              <-    3752    0.217    2.814  hill_final.py:94(best_solution)
hill_final.py:48(calculate_distance)                                     <-  397712    2.506    2.530  hill_final.py:64(neighbors)
             

Observamos que existe una fuerte interacción entre calculate_distances y neighbors. Algo que es sumamente importante mencionar y revisar es que existen llamadas vacías particularmente de las funciones anteriores.

> **Medición de tiempo, line_profiler**

Una vez realizado el perfilamiento con cProfile, nos enfocaremos en analizar las funciones que tienen un mayor consumo de tiempo utilizando line_profiler:

In [55]:
import line_profiler
line_prof = line_profiler.LineProfiler()
print(line_prof(hc.best_solution)(tsp_cities, 0 , 1e-9, 300))

(3319.127344323918, [0, 3, 8, 4, 2, 7, 16, 14, 9, 10, 15, 11, 13, 12, 6, 5, 1, 0], 5.007052659988403)


In [56]:
print(line_prof.print_stats())

Timer unit: 1e-06 s

Total time: 5.0023 s
File: /home/ubuntu/practica-2-segunda-parte-jlrzarcor/notebooks/hill_final.py
Function: best_solution at line 94

Line #      Hits         Time  Per Hit   % Time  Line Contents
    94                                           def best_solution(coordinate, initial_point = 0, tolerance = 1e-7, n_restarts = 100):
    95                                               """
    96                                               finds an optimal solution for the TSP problem using hill climbing algorithm
    97                                                   input:
    98                                                       points[array]: coordinates of the places to be visited 
    99                                                       initial_point[integer]: number of the place to be visited first
   100                                                       tolerance[float]: value that indicates the solution is not improving
   101                  

Nuestra línea de código que más veces se ejecutó es la correspondiente a nuestro segundo ciclo _while_, el cual determina la mejor ruta del vecindario para una _initial random solution_ con 1835 ejecuciones, lo cual concuerda con las 300 iteraciones de reinicio (Ciclo for) y los while anidados para determinar las mejores rutas de forma aleatoria.